In [1]:
from PIL import Image
import cv2
import numpy as np
import pandas as pd
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.models import load_model
import glob
import os

Using TensorFlow backend.


In [2]:
# def make_X():
#     X=list()
#     for i in range(1,501):
#         img_name=str(i)+'.png'
#         im = Image.open(img_name)
#         np_im = np.array(im)
#         resized_img=cv2.resize(np_im,(28,28), interpolation = cv2.INTER_CUBIC)
#         X.append(resized_img.tolist())
#     return np.array(X)

In [3]:
def make_y(ds):
    y=[]
    for i in range(500):
        y.append(ds[i])
    return np.array(y)

In [4]:
path="C:/Users/ankur/Desktop/captcha/data/"
path=os.path.realpath(path)
# im=Image.open(path+'/3.png')
def make_X():
    X=list()
    for i in range(1,501):
        img_name=path+'/'+str(i)+'.png'
        im = Image.open(img_name)
        np_im = np.array(im)
        resized_img=cv2.resize(np_im,(28,28), interpolation = cv2.INTER_CUBIC)
        X.append(resized_img.tolist())
    return np.array(X)
# xxx=make_xx()

In [5]:
df=pd.read_excel('y.xlsx')
y=make_y(df['y'])
X=make_X()
y=np_utils.to_categorical(y)

In [6]:
X.shape

(500, 28, 28)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:
len(X_test)

165

In [9]:
from keras.models import Sequential
from keras.layers import Dense

In [10]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [11]:
# flatten 28*28 images to a 784 vector for each image
num_pixels = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')

In [24]:
X_train.shape

(335, 784)

In [12]:
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

In [13]:
num_classes = y_test.shape[1]

In [14]:
# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
	model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [15]:
# build the model
model = baseline_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Train on 335 samples, validate on 165 samples
Epoch 1/10
 - 0s - loss: 2.0271 - acc: 0.3463 - val_loss: 0.7136 - val_acc: 0.9879
Epoch 2/10
 - 0s - loss: 0.5206 - acc: 0.9881 - val_loss: 0.2006 - val_acc: 0.9939
Epoch 3/10
 - 0s - loss: 0.1467 - acc: 1.0000 - val_loss: 0.1119 - val_acc: 0.9939
Epoch 4/10
 - 0s - loss: 0.0604 - acc: 1.0000 - val_loss: 0.0793 - val_acc: 0.9939
Epoch 5/10
 - 0s - loss: 0.0253 - acc: 1.0000 - val_loss: 0.0733 - val_acc: 0.9939
Epoch 6/10
 - 0s - loss: 0.0122 - acc: 1.0000 - val_loss: 0.0763 - val_acc: 0.9939
Epoch 7/10
 - 0s - loss: 0.0080 - acc: 1.0000 - val_loss: 0.0796 - val_acc: 0.9939
Epoch 8/10
 - 0s - loss: 0.0053 - acc: 1.0000 - val_loss: 0.0827 - val_acc: 0.9939
Epoch 9/10
 - 0s - loss: 0.0034 - acc: 1.0000 - val_loss: 0.0858 - val_acc: 0.9939
Epoch 10/10
 - 0s - loss: 0.0023 - acc: 1.0000 - val_loss: 0.0887 - val_acc: 0.9939
Baseline Error: 0.61%


In [16]:
res=model.predict_classes(X_test)

In [17]:
X_test[0].shape

(784,)

In [18]:
res[1]

8

In [19]:
y_test[2]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.], dtype=float32)

In [20]:
scores=model.evaluate(X_test,y_test)
print(scores[1]*100)

165/165 [==============================] - 0s 145us/step
99.3939393939


In [21]:
model.metrics_names

['loss', 'acc']

In [22]:
model.save('nn_captcha.h5')

In [23]:
# model = load_model('my_model.h5')